In [17]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

# Move the username, password, host, port, database, and collection specifications to this document
# to be passed into the AnimalShelter initialization function as parameters.
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 31782
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Specify the Grazioso Salvare image specification (within this same file)
image_filename = 'grazio-salvare-logo-medium.png'
# Open the image to place it onto the dashboard
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Set up the application layout
app.layout = html.Div([
    # Unique identifier at the top
    html.Center(html.B(html.H1('K9Recruit - Andrew Emilio DiStefano'))),
    html.Hr(),
    html.Center(
        # Place the aforementioned Grazioso Salvare image under the unique identifier
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    
    # This div contains the radio items to select which type of rescue to search canines for
    html.Div( 
        # Create the radio items for 'water rescue', 'mountain rescue', 'disaster rescue', and 'reset'
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            # Set the default value to 'reset'
            value='reset'
        )
    ),
    
    # Set up the data table
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        # Prevent issues with not selecting an initial row by specifying index 0 by default.
        selected_rows = [0],
        # Make the table editable.
        editable = False,
        # Enable the selection of single rows.
        row_selectable = "single",
        # Enable filters.
        filter_action = "native",
        # Enable sorting.
        sort_action = "native",
        sort_mode="multi",
        # Enable pagination.
        page_action = "native",
        # Set the first page to 0.
        page_current = 0,
        # Allow 10 entries per page.
        page_size = 10,
        
    ),
    html.Br(),
    html.Hr(),
     html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 ),
                 dcc.Graph(
                     id="pie-chart",
                     className='col s12 m6',
                 )
             ])
])
#############################################
# Interaction Between Components / Controller
#############################################
        
# Filters
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        # Set the criteria for 'water rescue'
        if filter_type == 'water':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            # Drop the _id' column to avoid including values of type 'ObjectID', which
            # will cause a TypeError and keep the interactivity from working properly
            df.drop(columns=['_id'],inplace=True)
        # Set the criteria for 'mountain rescue'
        elif filter_type == 'mountain':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            # Drop the _id' column to avoid including values of type 'ObjectID', which
            # will cause a TypeError and keep the interactivity from working properly
            df.drop(columns=['_id'],inplace=True)
        # Set the criteria for 'disaster rescue'
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever", 
                                  "Bloodhound","Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            # Drop the _id' column to avoid including values of type 'ObjectID', which
            # will cause a TypeError and keep the interactivity from working properly
            df.drop(columns=['_id'],inplace=True)
        # Set the criteria for 'reset' or other
        else:
            df = pd.DataFrame.from_records(shelter.read({
# Uncomment the following lines of code to set the starting and 'reset" state of the dashboard to
# depict only the dog breeds which are viable for at least one of the three search-and-rescue types:
#                "animal_type": "Dog",
#                "breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog", 
#                                  "Siberian Husky", "Rottweiler", "Golden Retriever", "Bloodhound",
#                                 "Rottweiler", "Doberman Pinscher", "Labrador Retriever Mix",
#                                  "Chesapeake Bay Retriever", "Newfoundland"]},
#                "sex_upon_outcome": "Intact Male",
#                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            # Drop the _id' column to avoid including values of type 'ObjectID', which
            # will cause a TypeError and keep the interactivity from working properly
            df.drop(columns=['_id'],inplace=True)
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
            
        
        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

# Update the pie chart based on user input
def update_graphs(viewData):
    dffPie = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dffPie, names='breed',)
        )    
    ]

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-id", "data")]
)
def generate_chart(data):
    dff = pd.DataFrame.from_dict(data)
    fig = px.pie(
        dff,
        names='breed',
    )
    return fig

# Highlight a row on the data table when it is selected by the user
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):

    dff = pd.DataFrame.from_dict(viewData)

    # If no index is selected,
    if index is None:
        # then use the latitude and longitude corrdinates for Austin, TX
        markerArray = (30.75,-97.48) 
        # Label the default location 'Austin Animal Shelter' in 'Austin, Texas'
        toolTip = "Austin Animal Shelter"
        popUpHeading = "Austin Animal Shelter"
        popUpParagraph = "Austin, Texas"
    # Otherwise, 
    else: 
        # create a dataframe with the data available within the selected row,
        dff = pd.DataFrame(df.iloc[index]) 
        # get the latitude value from the row within MongoDB (clean output to avoid errors),
        coordLat = float(dff['location_lat'].to_string().split()[1])
        # get the longitude value from MongoDB (clean output to avoid errors),
        coordLong = float(dff['location_long'].to_string().split()[1])
        # and set the latitude / longitude array with these latitude and longitude values.
        markerArray = (coordLat, coordLong) 
        
        # Label the breed of the animal at the shown coordinate.
        toolTip = dff['breed']
        # Label the name of the animal at the shown cooridnate.
        popUpHeading = "Animal Name:"
        popUpParagraph = dff['name']


    return [
        # Return a map of the specifications detiale below:
        dl.Map(style={'width': '800px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=markerArray,
              children=[
              # Label the breed of the animal at the shown coordinate.
              dl.Tooltip(toolTip),
              # Label the name of the animal at the shown cooridnate.
              dl.Popup([
                 html.H3(popUpHeading),
                html.P(popUpParagraph)
             ])
          ])
       ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:29172/
